In [1]:
#!pip install plyer
#from plyer import notification
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import geckodriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from datetime import datetime as date
import pandas as pd, numpy as np
import warnings as ws
ws.filterwarnings("ignore")
import re, os, time, glob
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.common.keys import Keys
import pickle
from typing import List
# Create directory if it doesn't exist
directory = "Fetched Data\\Search results"
if not os.path.exists(directory):
    os.makedirs(directory)


def clean_str(string):
    string = string.replace('\xa0', " ").strip()
    string = string.replace('\n', "")
    string = string.replace('\t', "")
    return string.strip()

In [2]:
# Initialize Firefox with options
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
geckodriver_autoinstaller.install()
firefox = webdriver.Firefox(options=options)

Firefox version:  117.0.1
Available geckodriver version:  v0.34.0


In [30]:
BASE_URL = 'https://cdscomdonline.gov.in/NewMedDev/ListOfApprovedDevices'
firefox.get(BASE_URL)
WebDriverWait(firefox, timeout = 10)

<selenium.webdriver.support.wait.WebDriverWait (session="2a77e8f9-8856-4800-b1d1-fee3804bd9a1")>

In [25]:
def extract_page_data(firefox):
    soup = BeautifulSoup(firefox.page_source, 'html.parser')
    data_table = soup.find('table',{'id':'example'})
    page_data = pd.read_html(str(data_table))[0]
    return page_data

In [31]:
page_counter = 1
collector = []
while True:
    print(f"Getting data - Page {page_counter}")
    page_data = extract_page_data(firefox)
    collector.append(page_data)

    try:
        # Wait for the next button to be clickable
        next_btn = WebDriverWait(firefox, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="example_next"]/a')))
        
        # Click the next button
        next_btn.click()
        time.sleep(10)
        page_counter+= 1
    except TimeoutException:
        # Break the loop if the next button is not clickable
        print("No more pages to scrape.")
        break

Getting data - Page 1
Getting data - Page 2
Getting data - Page 3
Getting data - Page 4
Getting data - Page 5
Getting data - Page 6
Getting data - Page 7
Getting data - Page 8
Getting data - Page 9
Getting data - Page 10
Getting data - Page 11
Getting data - Page 12
Getting data - Page 13
Getting data - Page 14
Getting data - Page 15
Getting data - Page 16
Getting data - Page 17
Getting data - Page 18
Getting data - Page 19
Getting data - Page 20
Getting data - Page 21
Getting data - Page 22
Getting data - Page 23
Getting data - Page 24
Getting data - Page 25
Getting data - Page 26
Getting data - Page 27
Getting data - Page 28
Getting data - Page 29
Getting data - Page 30
Getting data - Page 31
Getting data - Page 32
Getting data - Page 33
Getting data - Page 34
Getting data - Page 35
Getting data - Page 36
Getting data - Page 37
Getting data - Page 38
Getting data - Page 39
Getting data - Page 40
Getting data - Page 41
Getting data - Page 42
Getting data - Page 43
Getting data - Page 

KeyboardInterrupt: 

In [32]:
len(collector)

119

In [33]:
cdsco_approved_devices = pd.concat(collector)
cdsco_approved_devices = cdsco_approved_devices.drop_duplicates()

In [40]:
cdsco_approved_devices.to_csv("CDSCO_IN - Approved Medical Devices 4Jan24.csv")
cdsco_approved_devices.to_excel("CDSCO_IN - Approved Medical Devices 4Jan24.xlsx")

In [41]:
cdsco_approved_devices

,S. No.,License Number,License Holder Name,Device Name(Brand Name)-Intended Use,Approving Authority,Device Class
0,1,02/SCof2018,GPC Medical Limited,Urethral Catheter(GPC Urethral Catheter)-It is...,Uttar Pradesh,Class B
1,2,03-13-0966,TTK Healthcare Limited,Natural Rubber Latex Male Condoms(ULINZI - For...,"Central Drug Standard Control Organisation, So...",Class C
2,3,0499,QUALPRO DIAGNOSTICS A DIVISION OF TULIP DIAGNO...,25-OH VITAMIN D ELISA-The Product is intended ...,Goa,Class B
3,4,0623,CORAL CLINICAL SYSTEMS A DIVISION OF TULIP DIA...,ENZYMATIC ASSAY FOR THE QUANTITATIVE DETERMINA...,Goa,Class B
4,5,13/28/17,Dynamic Techno Medicals Pvt. Ltd,Surgical Suture(Dynalon)-Use in general soft t...,"Central Drug Standard Control Organisation, So...",Class C
...,...,...,...,...,...,...
49,10950,S0079/15/MB,VANGUARD DIAGNOSTICS PRIVATE LIMITED,HIV Gen.4 Microwell ELISA(VDx)-HIV Gen. 4 Micr...,"CDSCO -North Zone, Ghaziabad",Class D
50,10951,SC/02/2018,GPC Medical Limited,Locking Bolts 7.5mm Cannulated (Self Tapping)(...,"CDSCO -North Zone, Ghaziabad",Class C
51,10952,TN00002456,TTK HEALTHCARE LIMITED (ORTHO DIVISION),Total Hip Replacement System(Pl refer Accessor...,"Central Drug Standard Control Organisation, So...",Class C
52,10953,TN00002473,CARBORUNDUM UNIVERSAL LTD,ADHESIVE DRESSING BANDAGES FABRIC(CERDAK)-Exud...,"Central Drug Standard Control Organisation, So...",Class C
